In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import copy 

import seaborn as sns
sns.set(color_codes=True)

In [ ]:
from sklearn.model_selection import train_test_split

def split_dataset(data, n:float, seed=0):
    dummy_labels = np.ones(data.shape[0])
    X_train, X_test, y_train, y_test = train_test_split(data, dummy_labels, test_size=n, random_state=seed)
    return [X_train, X_test]


In [1]:
!pwd

/home/acevedo/Desktop/Current_GAN/MicrobiotaGAN/notebooks


In [ ]:
my_data = pd.read_csv('../data/DS_1.csv', header=None) .values

In [ ]:
gan_data = pd.read_csv('../data/gan_dirichlet_1.csv', header=None) .values

In [ ]:
train_set, test_set = split_dataset(my_data, n=0.2)

In [ ]:
print(np.sum(my_data ==0))
print(np.sum(gan_data ==0))

# Error de Validacion

En esta parte hay que definir un error para nuestra GAN.

## Clasificacion 
+ Para problemas de **clasificacion** una medida del error puede ser **clasificaciones correctas/numero total de clasificacones**
+ Para problemas de clasificacion multi clase existe una perdida logaritmica definida como: $$\frac{-1}{N} \mathop{\sum_{i=1}^{N}\sum_{j=1}^{M}} y_{ij}*log(p_{ij})$$

Donde:
+ $y_{ij}$ indica si la muestra i pertenece a la clase j
+ $p_{ij}$  es la probabiliad de que la muestra i pertenezca a la calse j 

+ **F1 Score** : $$2 \times \frac{1}{ \frac{1}{precision} + \frac{1}{recall}}$$. Precision y recall se definen en terminos de TruePositives, FalsePositives y FalseNegatives

## Regresion

+ Mean Absolute Error
+ Mean Squared Error

## Medidas para las GANS

En el caso de las GANs, la funcion objetivo o de perdida mide que tan bien el generador engaña al discriminador. Entonces, esta no nos dice mucho acerca de que tan reaslaticas o diversas son las muestras generadas por la GAN.

Para el caso de **GAN de imagenes**, hay dos medidas del rendimiento de una gan, el Inception Score y el Frechet Inception Distance. (https://medium.com/@jonathan_hui/gan-how-to-measure-gan-performance-64b988c47732)

**Una buena GAN es una que genera muestras diversas y realistas**. En el caso de las imagenes, no es trivial hacer que una computadora te diga que tan realistas es una imagen.

# Adapted Inception Score

In [ ]:
import sys
sys.path.append("../")
import scipy
from sklearn.preprocessing import normalize
from numpy import inf
from copy import copy 

import warnings
warnings.filterwarnings("ignore")

from get_disc_score import get_disc_score

In [ ]:
model_path =  "../model/dir1/my_gan.ckpt"

def pseudo_is_score(data, restore_path):
    x = get_disc_score(val, restore_path )
    
    logits = np.exp(x)
    
    py = np.mean(logits, keepdims = True)
    
    # pxy_k = logits[k,:]
    
    print(py.shape)
    print( logits.shape)
    
    assert py.shape[1] == logits.shape[1]
    
    N = data.shape[0]
    
    #pre_score = [DKL( logits[k,:], py ) for k in range(N) ]
    
    #pre_score = np.array(pre_score)
    return py, logits

    

In [ ]:
val = my_data[1:5,:] #.reshape(1,100)

py, logits = pseudo_is_score(val, model_path)
py = py.flatten()

In [ ]:
a = logits[0,:]

b = py

print(a)
print(b)

DKL(a,b)

# Directly use DKL

In [ ]:
def DKL(pk_, qk_, epsilon = 0.01):
    
    """   
    n = pk.shape[0]    
    normed_pk = normalize(pk.reshape(1,n),  norm='l1')
    normed_qk = normalize(qk.reshape(1,n), norm='l1')
    print(normed_pk)  
    """
    pk = copy(pk_)
    qk = copy(qk_)
    
    #print(pk.shape)
    a = np.sum(pk ==0)
    b = np.sum(qk ==0)
    # assert a ==0 and b ==0
    
    #if  a !=0 or b !=0:
        #print("Padding Zeroes")

    # Beware  zeros
    pk[pk==0] = epsilon
    qk[qk==0] = epsilon
    
    return scipy.stats.entropy(pk,qk)      

In [ ]:
print(my_data.shape)
print(gan_data.shape)



In [ ]:
gan_data.mean(axis=0)

In [ ]:
n =10000
a = gan_data[0:n,:].mean(axis=0)
b = my_data [0:n,:].mean(axis=0)

print(np.sum(a ==0))
print(np.sum(b ==0))

DKL(b,a) == DKL(b,a)

DKL(a,b)


In [ ]:
a = gan_data[0:n,:]
b = my_data [0:n,:]

print(np.sum(a ==0))
print(np.sum(b ==0))

In [ ]:
entropy = [DKL(a[k,:], b[k,:]) for k in range(n)]
entropy = np.array(entropy)
np.sum(entropy== inf)

In [ ]:
entropy 

In [ ]:
k = 0
#print(a[k,:])
#print(b[k,:])
#print(scipy.spatial.distance.euclidean(a[k,:], b[k,:]))

print(a[k,:].sum())
DKL(a[k,:], b[k,:]) 

In [ ]:
#
#entropy[entropy == inf] = 0

from scipy.spatial import distance

In [ ]:
index_list = []

for k in range(10000):
    if entropy[k] == inf:
        index_list.append(k)

In [ ]:
k = index_list[20]


DKL(a[k,:], b[k,:]) 

In [ ]:
for e in range(len(index_list)):
    k = index_list[e]
    print(distance.jensenshannon(a[k,:], b[k,:]))

In [ ]:
k = 87

print(a[k,:].std())
print(b[k,:].std())
print(a[k,:].sum())

DKL(a[k,:], b[k,:]) 

In [ ]:
r = distance.jensenshannon(a, b)

In [ ]:
c = b + np.random.normal(0,1,100)

In [ ]:
 
entropy2 = [distance.jensenshannon(a[k,:], b[k,:]) for k in range(n)]
entropy2 = np.array(entropy2)


In [ ]:
entropy2.mean()

In [ ]:
entropy4 = [DKL(a[k,:],b[k,:]) for k in range(n)]
entropy4 = np.array(entropy4)

In [ ]:
entropy4.shape

In [ ]:
temp = entropy4[entropy4 < 1E308]#.mean()
temp.mean()

In [ ]:
c =  b +0.1*np.random.normal(0,1,100)
c.mean()

c =0.8*(b+0.9)
c.mean()

In [ ]:
entropy3 = [DKL(a[k,:],c[k,:]) for k in range(n)]
entropy3 = np.array(entropy3)

In [ ]:
entropy3[entropy3 < 1E308].mean()

In [ ]:
temp = entropy4[entropy4 < 1E308]#.mean()
temp

In [ ]:
print(temp.shape)
np.mean(np.exp(temp))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(color_codes=True)

%matplotlib inline

In [ ]:
sns.distplot(temp)